In [ ]:
import numpy as np
import pandas as pd

import os
import flask
from flask import *
from werkzeug.serving import run_simple

In [ ]:
# Connect to the database and setup the app
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

In [ ]:
df = pd.read_csv("df_pred_ext (1).csv").drop(["Unnamed: 0", 
                                            "job_order", 
                                            "source", 
                                            "time_between"], axis=1)

In [ ]:
df.head()

In [ ]:
df_grouped = df.groupby("candidate_id")

In [ ]:
# example = df_grouped.get_group(84612)

In [ ]:
# example = example.reset_index().drop("index", axis=1)
# example.index += 1
# example = example.T

In [ ]:
@app.route('/', methods=["GET", "POST"])
def index():
    if request.method == "POST":
        session["user_type"] = request.form['submit_button']
        session["example"] = 0
        return redirect(url_for("sliders"))
    else:
        return render_template("/index.html")

In [ ]:
@app.route('/sliders', methods=["GET", "POST"])
def sliders():
    
    if session["example"] >= 3:
        return "klaar!"
    
    # instead --> get examples[group_type][example_number]    
    df = df_grouped.get_group(np.random.choice(
        list(df_grouped.indices.keys()))
                             ).reset_index().drop(["index", 
                                                   "candidate_id"], axis=1)
    
    df.index += 1
    df = df.T
    
    return render_template("/sliders.html", dataframe=df.to_html(classes="table"))

In [ ]:
@app.route("/store_results", methods=["POST"])
def store_results():
    
    r = request.get_json()
    values = dict(zip(["isco code", "company name", "education", "skills", 
                       "Licenses", "Languages", "Other stuff"], r["values"]))
    
    print(session["user_type"], session["example"], values)
    session["example"] += 1
        
    return redirect(url_for("sliders"))

In [ ]:
run_simple('localhost', 8080, app)